In [1]:
# Here we set up the loan ids for the development dataset and testing
# Should try and periodically resample the dev_ids every year

In [1]:
import sys
import os
import pickle
import pandas as pd
import numpy as np
from pandas.api.types import is_string_dtype
import math
import re
from tqdm import tqdm_notebook, tqdm
import j_utils.munging as mg
from lendingclub import config

pd.set_option('max_columns', 999)

rli = pd.read_feather(os.path.join(config.data_dir, 'raw_loan_info.fth'))
# Lendingclub still has internal grades through G
# take a random sample of 100000 loans to work with
# rli = rli.sample(100000, random_state=42)
dev_ids = rli['id'].tolist()

In [2]:
# # define those dev_ids here
# must_include_ids = [12287070]
# # dev_ids is random sample so set and then list to remove possible dupes
# dev_ids = list(set(must_include_ids + dev_ids))
# print(len(dev_ids))

In [3]:
dtypes = {'LOAN_ID': int,
 'PBAL_BEG_PERIOD': float,
 'PRNCP_PAID': float,
 'INT_PAID': float,
 'FEE_PAID': float,
 'DUE_AMT': float,
 'RECEIVED_AMT': float,
 'RECEIVED_D': str,
 'PERIOD_END_LSTAT': str,
 'MONTH': str,
 'PBAL_END_PERIOD': float,
 'MOB': int,
 'CO': int,
 'COAMT': float,
 'InterestRate': float,
 'IssuedDate': str,
 'MONTHLYCONTRACTAMT': float,
 'dti': float,
 'State': str,
 'HomeOwnership': str,
 'MonthlyIncome': float,
 'EarliestCREDITLine': str,
 'OpenCREDITLines': float,
 'TotalCREDITLines': float,
 'RevolvingCREDITBalance': int,
 'RevolvingLineUtilization': float,
 'Inquiries6M': float,
 'DQ2yrs': float,
 'MonthsSinceDQ': float,
 'PublicRec': float,
 'MonthsSinceLastRec': float,
 'EmploymentLength': str,
 'currentpolicy': int,
 'grade': str,
 'term': int,
 'APPL_FICO_BAND': str,
 'Last_FICO_BAND': str,
 'VINTAGE': str,
 'PCO_RECOVERY': float,
 'PCO_COLLECTION_FEE': float}

In [4]:
# to make pmt_hist_skiprows from dev_ids. Skiprows is fastest way I've found to read this df
# 2019-11-26 11:43:15 takes about 53ish seconds
pmt_hist_fnames = [f for f in os.listdir(config.wrk_csv_dir) if 'PMTHIST' in f]
reader = pd.read_csv(os.path.join(config.wrk_csv_dir, pmt_hist_fnames[0]), chunksize=25000000, iterator=True, na_values=['*'], dtype=dtypes)# 2500000 best so far
pmt_hist = pd.concat([chunk for chunk in reader])

In [5]:
# what is MONTHLYCONTRACTAMT
# change 'Inquiries6M': dtype('float64'),
# 'PublicRec': dtype('float64'),

In [6]:
skip_pmt_hist = pmt_hist.query('LOAN_ID not in @dev_ids')
# +1 because the rows need to be shifted to account for first row as header
pmt_hist_skiprows = list(skip_pmt_hist.index.values + 1)

In [7]:
len(skip_pmt_hist)

0

In [8]:
# for testing
# what is wrong with pmt_hist_skiprows? try loading from pmt_hist_skiprows
# and compare to subset_pmt_hist
# subset_pmt_hist = pmt_hist.query('LOAN_ID in @dev_ids')
# subset_pmt_hist = subset_pmt_hist.reset_index(drop=True)
# print('loading pmt_hist; skipping {0} rows'.format(len(pmt_hist_skiprows)))
# ss_pmt_hist = pd.read_csv(os.path.join(config.wrk_csv_dir, pmt_hist_fnames[0]), skiprows=pmt_hist_skiprows, na_values = ['*'], dtype=dtypes)

In [10]:
# save the pmt_hist_skiprows
with open(os.path.join(config.data_dir, 'pmt_hist_skiprows.pkl'), 'wb') as f:
    pickle.dump(pmt_hist_skiprows, f)

In [11]:
# save dev_ids
with open(os.path.join(config.data_dir,'dev_ids.pkl'), 'wb') as f:
    pickle.dump(dev_ids, f)

In [12]:
# save dev_ids
with open(os.path.join(config.data_dir,'pmt_hist_dtypes.pkl'), 'wb') as f:
    pickle.dump(dtypes, f)

# old stuff

In [ ]:
# dev_ids = [135846826, 126461631,  79581442, 124857628, 140938251, 128227825,
        77350855,   1622294,  94750460,  40440155,  58460477, 122236264,
        58310451, 108815122, 117272801,  58723912, 147098010,   7439256,
       137750089, 146504821,  86101283, 144364154,   1501323,  90096681,
        67848659,  57853116, 111569907,  63516265,  38589235,  92577636,
        41288227,  67266298,  58722697, 133780807, 105413345,  44945334,
        96835618, 125050100, 132023104, 128356051, 152254724, 134600581,
       124044992,  40450344,   1096267,  69694504, 112061999, 132489437,
       115096387,  75859145,   3369311, 145439445, 147859767, 137363962,
        68192137,  66666852, 127340822,  61922528, 102576688,  56019293,
        58311146, 136740571, 149434547,   6168240, 134782210, 151500235,
        92331652, 124133601, 131108991, 109081138,  43580089, 120198017,
        68845606,  93653072,   3675519, 153301478, 148709078,   3163681,
        21490500, 137497179, 128298890, 105293137,  68617014, 116708031,
        57034995, 126441771,  91843262, 147486222,  96879243, 149450336,
       125302795, 121221062, 134878138,  50536945,  33311596,  92471041,
        79682661, 147752604, 123600756,  96512033, 143533032,   1954888,
        37791598,   6180874,  75163117,  94107762, 146677971, 137749683,
        55091269, 111128383, 143710139, 152148404, 128950358, 143850957,
       135938469,  40362823, 125893511, 108135466,  59421826, 111739231,
        15601414,  38271302, 133436749,  23663205, 131099187, 121816007,
       150854561,  54313330,  53413670,   1610024, 140286292, 104654409,
         1398496, 138013163,  50596482,  66150344, 152097267, 150241590,
       152917631, 137616978,  43490849,  35104542,  85891771,  78361188,
        68563262,  60395247,  75718409,  19617865,    744425,  74701633,
       113840570, 151551562, 137358785,  22273398,  92450784, 110815468,
         1522586, 116261696,   7657770, 136535442,  96768843, 150069460,
        62824432,   5146076,  96719844,   9228749, 110366922,  66665470,
         1481918,  39360022,  79270633, 145961362,  64058451,  78348523,
        66250832,  39532747,   3537183, 143576265,  43519789,  31527635,
        55920422,  53161243,  10136622, 135929344, 103628614,  68112789,
        20117349,   6527892,  28913254,  38048288,  39700515, 143781992,
        53944353, 141734062, 114220408,  77238440,  94011838, 150022370,
        89031370,  55372178, 141645418, 146655577, 144465272, 136911077,
        85602364,  72543758, 110749842,  85653589,   8796277, 128142105,
        65293186, 114150438, 113569433, 141642528,  91122517, 152229158,
          846512,  75121727,   6307120,   1521345, 142338777,   3239917,
        91364922,  52321481,  92104632, 107206105,  51877344,   6295774,
        73421427, 124813136,  78011383,  12487788,  77391376, 102116349,
        15270225,  70905022,   1611036,  67299012, 112810780, 141680696,
       146858550, 141932918, 113245879, 147894918, 119826319,   1436191,
       143264702, 126614074, 129068882,  95062210, 150217288,  81454544,
       105938686, 112145333, 124035288,  95038457,   4255623,  31828305,
        12616498, 152637927,  38616032,  17243501, 102156152,   2300718,
         1596444, 143406612,  20368270,  53019672, 152863960,  91964214,
       150267047, 150789785,  98928320, 148478059,  56002997,  86603747,
        60597235, 147402061, 131470719, 152488194,  91029372, 152042277,
       109634972,   1038484, 130375100,  49943458, 143291480,  50103421,
        94417750,  57236280, 110022457, 135248326,  72733917, 141066526,
       139790400, 128930067, 106630457,  39521435, 127913420,    753608,
        63336866,  75849371,   7604816,  95815945, 149848137,  16151984,
       102495735,  50578693,  58150712, 136066048,  36240297,  75283620,
       143425819, 138512104, 126700126, 126364351,  66520291,  91153016,
       124917275, 146301309,  30736190,  51857314,   1557909,   3498463,
        77159327, 119704495, 153368273,  33521749, 130500456,  68616060,
        20548653,  42313562,  57672472,  63357147,   6306795,  59829726,
       113182840,  38922329,  17814818,  98907578,  26408800,  75982872,
       152634570, 147610211, 139108511, 123052809,  40277718, 105253485,
        13076236,   1502741,  60356746, 148496681,  99942588, 127489411,
        42404824,  22411880,  34353777, 149012131,  40931182, 131974737,
       137576617,   1373438, 135828864, 107081611, 152974170,  96995279,
        88724888,  91171134, 141478713,  60355827,    490429, 128378929,
       139934021,  46467452, 152720655,  55390268, 120008684,  94394178,
        82860114,  46804170, 149682457, 138426532,  96488778,  12467721,
       111857741,  55288170,  56612154, 151842288, 112209400,  61339706,
       136332946,  27720690, 146744193,  17695703,  19997494,  93454961,
       106509935,  15469766,  83819871,  70553619, 144182394, 139116000,
       144626106,  70221856, 132031594,   1585986, 151223598, 137167746,
        28943423,  76202520, 122969871, 124072372,  97534895,  63813412,
        40443126, 109658201, 147988226,  25547252,  92221195, 133940164,
        51887878, 134532082, 130530531,  78749489, 147438343, 150520837,
       151421900,  11375637,  85952805,  57693510, 111758423,  65697184,
       152737715, 141697640,  96877721,  90210092, 149807063, 137358685,
        83172129, 116474821,  80730943,  92658519, 142873136,  50879287,
        62003228, 134356291, 127093239, 122350581, 142243530,  84364571,
       102484963, 141605677,  47604662,  77861920, 126172041, 125920896,
         1624054,  73398419, 132982792, 130609334, 146340796, 142110012,
       147492398,  14618679,  11334603, 129908674, 129941262,  65755982,
       132893154, 145640958,  61421025,  39532297, 127677886, 118552791,
        76915439, 146420924,  92524249, 108771274,  91565174, 102443337,
       115109925, 120189681,  42496019,  55211148,  43096325, 141548070,
       125289729,  11005022, 147943031, 144627107,   1007711, 133732197,
       150232609, 125204515,  65756254, 124433200,  89816724,  12346651,
       141082648, 131554564,   5254887, 130504053,  12585456, 124818924,
       121845715, 124852377,   5875972,  43420534,  39937999, 147194647,
       148610585, 144272171, 115055193, 143534648,   7945939, 130400755,
         5706091,  95334047,  88976478,  50758910, 132903518, 125686558,
         2475229, 130925131,  96886118,  66120188, 147831566, 134944278,
       151495864, 103517470,   8935842,  57136659, 148763835, 131158472,
        58692618,  70562718, 116819797,   9058332, 103708060,  88115334,
       145124444, 112049441, 148404947, 129593241,  92843610, 137445291,
       111580250,    759281,  74541874, 152036398,  94265351,  70992599,
       134677804,   1538267,  42484858,  86974187,   6720149,  61462130,
        74575247, 120562543,  33281867,  74525270,  61362250,  75568546,
        86913889, 125173856,   1293277,  68443401, 134149221, 145740765,
       127561979, 143517048,   3067066,  21130567,  31227246,  90885281,
       131625637, 129633513, 141027189,  42364764,   6729642, 151015775,
        51095168, 148607775, 116826278, 151500124, 134544076,    405857,
        63526612,  88495602, 149220163,  84689449, 146457604,  61412516,
        95333001, 117205235,  13018835,  67436139, 120255756, 132787478,
       148310733, 132458219, 132853601,   6168536, 146712607, 151017265,
       120921832,   3290060,  67015116, 147948800,   6158392,  84635462,
       149181383,  94626915, 122191689,  47604440,  85942157,  39721865,
       136436691, 130848714, 103485417, 109048596, 127162959, 147798831,
         5787205, 143116450,  46114358,   9037424, 153896237, 144817593,
         6307623, 104250264, 111955178,  76373598,  75112841, 149873164,
       138362892,  48203285,  76436566, 124085512,  38506119, 112735784,
        12254919,  95140314, 136442461,   6265170, 146254300, 111778794,
       133387270,  46685310, 134480766,  89033569, 131370199, 131688024,
        59081139,  27090010,  94328755,  81366111,  54178743, 140499473,
       123961231,  50417716,  71736428,  63374420,  67237716, 101539270,
        88990836, 151305778,   5445425, 109650267,   3152912, 139674355,
       113355959,  14520629,   9206003, 149263240, 152354799,  76231215,
        66614669, 112030219,  46714766,  27690589,  24725910,  11666566,
        93685167, 133156701, 144689701, 145176637,  84604314,  50313564,
       147415054, 121863769,  65713983,  51958333,  48745964, 140446813,
        16702461,  69542755,    682652,  95030738, 139203997,  96853238,
       139250647,   6536493,   3057132, 134200048,  46588252,  93737721,
        96702432,  72682913,  15700072, 126629590, 144506564, 110367457,
        91632137,  57184268,   1105884, 147401258, 147005077, 135813908,
       125094194, 119364966,   4535309, 131529742,  84486529,    583564,
       119230221,  63184426,  55240686,  31306350,  91533155,   4195709,
       151904533, 152758787, 116683934, 151033421,  93980289,  14669949,
       147700451, 103759521, 130867991,   3700078, 151470808,  11014894,
        17543161,   9855902, 147222997,  61342549,  73181459, 142984756,
        41070375,  99886116,  65170008,  49883624,  96765185,  28712700,
        96243104,   4525027,   1497132,  97963444, 105163147, 143014317,
        69635570, 143220452, 114163328, 148029590,  73529014, 150109262,
       110362691,  19647274, 148783537,  63287300,  12438514,  95983815,
        99828939,  83133081, 149472753, 122349652, 132730528,   3154328,
        85156812,  13077800,  57296705,   5042059,  55241402, 149802655,
       110794019,  60236457, 146095088,  23893765, 147919432,  60245302,
        24916010,  76448293, 127376042, 115053333,  42673687, 142192250,
       146842446,  13487251,  22351585,  77109538,  97303240, 131707868,
       142279262,   1551470,   1605448,   4465581,  96916576,  60772136,
         6915845, 106815135,  44835252,   6326339,  12257384,  96503556,
        90365119,  30946091, 130978495,  61513387, 139100648, 153973831,
        77765773,  88172679, 150859680,  97579931,  84324239, 145676778,
       150330851, 143314009,  68519139, 122652030,  91047873, 121332054,
         1453763,  67808367, 102393668,    412331,  34443234,    374295,
        13017406,   7944760, 104191657,  81366632, 148282488,    584774,
        63426509, 150679376,  80719820, 119153267, 136921284, 127537792,
        46255850, 138152576,  19255840, 143013321, 105689508,  84605524,
        98033950,  61732712,  95117219,   2084013, 115434320,  76588970,
        74102056,   3845130, 137659920,  97819303,    651219,  68580101,
       125307062, 149151373,  64300357, 147657272, 100016767,  46651398,
        46350465,  94192645,  84243505, 113710788,  89935275, 146152062,
       126695341, 124019233,  62884762, 138384776, 118427743,  96261264,
       142983951,  38698955, 148692017,  68415432, 135832671, 153468944,
       130693869,  35166150,  34983855, 112054618,  83284896, 118266455,
        63176647,  83825954,  68843282, 125142729,  45714357, 118331894,
       138057273, 140924622,  79672689,  10568180, 148018337,  77170561,
         5856111,  61423667,  12516945,  75165364, 125856934, 125231349,
       113912298,  12796296,  70582434,  57852188,  34963948, 146851953,
        89025115,  90991987,  46642711,  69594197, 132496649, 134050869,
        29704934,  92400944, 128405177,  88076712,  85555873,   5234954,
       132930263,  39532910,  12965836,  57125915, 147061220,   6875581,
       141477305, 115381620, 129216049,  93962260, 153164647,  56632965,
         6519046, 121372838,   1682212, 133599392,   1486539, 127064454,
        61962333,  12655541,  96468568, 143587019, 146945062,  51186139,
        99786466, 143404332,  86432271,  12529153,   6625310,  79211240,
         4376292, 122294499, 151357749,  34332618,  48724528,   9165183,
        63889967,  46456921,  24795966,  57448194, 143431390, 138770066,
       149641724, 145205745, 113811273, 138287637,  32129365,   7656307,
        91672129,   6156789,  68685141,  27722273,  75273451, 136227807,
        99242655, 141460622,  27400350,  94297298,  70632963, 141995283,
       142320632, 148135734,  83837196,  46735923,  70781677,  96884324,
       126179318, 148489055,  44299533, 147725037,   7618239,  14127829,
        85592468,  23293189, 126059660,  16091428,   8605520,  81443809,
        63702218, 107509853,  61354518, 152610126,    209667, 108054678,
        45604264,  20781551,  28563388,   5793395,  60814436,  83846250,
       119901385,  12567174,  20789507, 120083150,   7015765,  77675409,
        42275220,    605582,  88045478,  26378280,   6636284,   3631516,
       131168647, 128932654, 118664761, 130196390, 120040222,  12925764,
       153639119, 131432633,  90915971,  57236488,  10474910,  39612044,
        36450498,  46316291,  13577686,  65190208,  63989807,   1583297,
       148463976,    566233, 141495155, 124908655,  27040002, 137707240,
       124105589, 110257401,  96939176,  44926067, 145053330,  54778735,
       126192670,  89034559,  93996360, 128639215,  31386867,  90320895,
       145312724,  95149107,  75758606, 113548407,   3455379, 126158449,
        16010747, 122247832, 149395106,  36271625, 134391825,  34633450,
        68415437, 147443537,  69594039,   6538192, 138170981,  43459178,
       138438867,   3351279,    458675, 119418197, 123934732, 150731591,
        23123351,  29114504,  65611234, 135026486,  96983394, 138843822,
       107156563,  65151154, 142477229,  10082340, 142184017,  74683495,
        14448175,  87374368, 113127582,  61479347, 111845903,  11627192,
         3299450, 146818047,  61542792,  23473523,  32319356, 129617450,
       122968943, 132236115, 123045979, 135903639, 132725610,  69502537,
        80689592, 144994676,  75869761,  97933706, 109751473,  93197598,
          388426, 109340802,   9838294,  14568201, 131850539, 128896646,
         5864687, 143190281, 150154805,  65926859,  85126377,   1389238,
          544619, 130250935, 137802569, 138449969,  38597168,  49874554,
        38427748,  40921940, 120601811,  96984443, 116712262,  55308592,
       130935630, 139935898, 124795232, 131494182,  54334385,   1053321,
        77554388,  94360473,  12957399, 142812648,  50064742, 146555067,
       151399422,   3847696, 145707473,  59501924, 147843405, 129377300,
       140106082, 131474943,   1543032, 105969465, 113486372,  85892486,
       104016086,  55211563,  27702286, 110772458,   2089410, 145775174,
          174377,  81526235,  73687144,  97607088, 136321660, 131377094,
       153037707,  27711614, 104190431,   2295353, 126692544,  12677213,
        96930898, 148543414,   6605574,  48464418,   4797323,  68812616,
       138261445,  97554780,  22462696,  51406234, 131724850,    602852,
        99776563,   9035144,   1446072,   1045401,  14408066, 129379819,
        84556977,    771034,  12986916,  19836853, 111896450,   2475638,
       153430120, 113710838, 104724154,  32458893, 146957844, 141299159,
        57166899, 139051851,  92830812, 119990692,  96862913,  76215823,
       146961536,  41790271,  52829065,  31597754, 135538646, 128913612,
        76311575,  22342387,  14779086,  79210988,    481468, 134180075,
        47128668,  96731976, 148586093,   3357903, 121406132,  30185717,
        68404508, 147900631, 140112911, 140744648,  73734987, 151380752,
        68814700,  83274490,  19186784, 108441591,    707735,  98127711,
       151410063,  36128886, 115400353,  35226790,  73355960,  97874579,
       134417145,  70943791, 132259847, 148256391,   8994885,  40511705,
        62549645, 144786384,  41308624,  95756828, 151545918,  90138830,
        76022960,  11704568, 123989472, 131512856, 133442033,  81486337,
        66350156,   3527004, 144407057, 143103055,  65926988,  36118893,
        53904431, 129870687, 140239265,   2085826, 135210492, 153702369,
         5065498, 127215210, 100087753, 137879817,  23503284, 148165306,
        61409721,  18684811,   7450505, 131065606, 112938834,    500743,
       143496115,  79672307, 102106853,  58522944,  91668112,   3107744,
        90330931, 115096893,  68995515, 143610215, 124433550,  68781734,
        37988310,   6787510, 144393047, 113123318, 111885282,  37197495,
         1338443, 127611401,  71767332, 134827163,  65787149,  69614797,
       126778533,  44037353,  91677093,   1303959,  96603376,  73764784,
       148429462,  90194155, 125057723,  88764054,  61344289, 119696090,
        78119175,  10159325, 146590175,  41236984,  23713230,  92575926,
       102407266, 147380760, 124456074, 151681118, 146601231,  39028197,
       118458054,  30776161,  87968485, 147891318, 102714265,  68851979,
       147307452,  88772916, 135362424, 130147563, 148553227, 132653594,
       124731619,   1157398, 128027816, 119977597,  75587212,    454687,
         6889327, 143434513,  88976608,  53794960, 133331238,  76062580,
       113684463,  90756447, 142087892,   7687256,  50437103, 150915385,
        72215360,  75528016, 124085005, 126527428, 146706059,  59123016,
       127478047,  48454657, 137867695, 154121232,  69083100,   8145491,
       129136542,  74975895,  92411005,  97012817,  41063322, 129403781,
       136942915,  99350719,  84998381, 145297539, 146931264, 137570421,
       141682058, 139577347, 150052095,  72592901, 112730956, 104521212,
       134767683,  79021683,  61468147, 138914691,   5835104, 136160455,
         5638564,  11045036, 136531503, 104441616,  12977874, 143198525,
          509078, 126121319, 136484093,  61480749, 147021745,  61379713,
        17633629, 113913850,  55158882, 147960058, 139234643,  25357968,
        40430412,  95008155,  47979921, 150007057, 152752889,  44825194,
        77507266, 104472538,  68589938, 152365645,  94097445, 127677918,
        51225144, 142574141, 143569091,  59241839,   3151368, 132510803,
       121890703,  90266981,  77981897, 136301853, 105943087,  75190496,
        85535412,  28082303, 150573887, 148559951,   7337511, 146131537,
        85548732, 146231805,   5204675,  51977037,  10168665,  61972926,
        11956366,  93920440, 146976549,   1359726,  88095811, 148369819,
       142545612,  55280915, 150648980,   3240432,  97493334, 126780712,
        65181173, 112049615,  84024664, 110509510, 126521304,  16181649,
       129200180,  93493403,  93135610, 118310290,  10107570,  56652636,
        95112532, 139789160,  83679411,  58623216,  63386665,  72193333,
        60576715,  37631055,  75397267,  93109421, 122206584,  91132078,
         7936524, 152186251,   3734536, 140883659,  43945492,  62053169,
        46671498, 149652483,  29223885,  64027802,  94831259,  88665613,
       134816812,   6538409,  39549962, 143899701, 100158036,  45524691,
        94464502,  67277551,  26188045, 139031956,  51407448, 111416119,
       144687647,  76282689, 147254261, 150665815,  29213805,   7286024,
       130849492,  84477623,  61361417, 142632650,  88997034, 153752478,
         8989546,   6115002,  96289156,  60930284, 139669732, 149086649,
       123622008,   8514663, 130318977,  84044879, 134140970, 112299390,
        75135002,  90856892, 154043564,  95805781, 136407834,  46034154,
       138924860,  38584334, 123900746, 111601223,  60804663,  22422308,
        96275919, 148871460,   8616937, 143979933, 126911978, 122214629,
       151898426, 137665955,  32520095, 125327070,  66170151,  61513969,
        37297184,  99360822,  96416453, 148665207, 122232815,  90066821,
       145848822,  65655759, 139925436,  48624678, 119336964, 143857344,
       127299181,   5354802,  46620419, 118620290,  42274433,  46348638,
       149852066,  98907465,  60617536, 143618434,  68968249,   9238289,
       126725925,  21200242,  70996599, 136069390, 103197390, 127025463,
       126697571, 148884849, 146327744, 137206606, 143013263, 110649172,
        46225080,  67745452, 128974738, 145955794,  17342951,   6543225,
       123990409, 147550993,  93709338, 120262685,  51976652, 144712684,
       125037512, 108969084,  60545652,  48734812, 119361346, 124060242,
        80708655,  65835562, 152858441,  26700083, 135082818,  84800052,
       116893884,  66965172, 134356108,  40940631, 108772221, 123988877,
       125262725,   8355132,  73713899, 149138394, 145673392,  60913700,
        13117387, 146967290, 123619535,  38486375,  69196909,  60801280,
        67386122,  46809822, 104630039,   8567302,  51357276, 111699205,
        41630243,  33341166, 135902082,  85400114, 137886159,  88862443,
        77261133,  65422778,  57175893, 142983707,  28933927, 138131686,
       133830092, 142446867,  22274349,  44075832,  63194443,   8981689,
        72734476,  11355874, 139920820, 149493449,  75229395,   5474622,
       117199835, 115097607,    619805, 116586736, 100458578, 138857336,
        65984490, 131710002,    997734,  74774536, 121268273, 145624835,
        67285497,  65977080,  25977224,  78709272, 119302390,  77554778,
       110714633,  63389191,  39679336,  71894444, 149262498,  54435134,
        28652861, 139966534, 136921397,  67979369,  12386724,  68424618,
       153870570,  50581029, 152749079, 116095131,  20108954,  96676196,
        97559012,  65473236, 137580859,  20418746,   6567342,  57286988,
        98718560,  43996884,  63450166,  63430347,   8999975, 117890160,
       141106034, 134249230,  56592181, 112746040, 148979652, 149563030,
       126230765,  55202062, 139615043, 147144081,  73924798,   1097808,
        85028020, 146875262,  13366809, 118602004,  75174485,  70061097,
        43315572,  19946910, 106495640,  91102585, 112888349,   1138170,
        59481764, 151280400,  98084616,  89886667, 122301027,    519200,
        30326039,  22292335,  60914015,  90099821,   2365940, 134889248,
        59552521, 153123200,  11404873, 150142289,  65527505, 136399873,
          972373,  50435213, 111035936, 132050464, 116627683, 126248296,
       103213367,  40972029,  91447026,  44815484,  27521797,  77020661,
       148001541,   8959539,   5155797, 113769302,  15179114,  77901514,
       151425070,  47128651, 105939456, 149279400,    812148,   3197134,
        55072248,  12508957,  88859516,   5779098,  46640242,  20681697,
         5759001,  43290074,  66624682, 136034513,  95161531,  83364568,
        81567391, 125766895, 107079477,  71107784,   3350592,  12407057,
        40382610,  44479195,  19436291, 108237208,  40452046,  38540036,
        96083937,  41740760,  68407113,  61541001, 150736123,  79113859,
       140980078, 153225841, 146195115, 133207394, 142470152,  63752123,
        57388497,  88015585,   8564907, 118604057,  64161482,  61419578,
        63416897,  65705683,  17745778,  73282615, 109101523, 150770670,
        14588298,  43509115,  50053972,  86042699, 138966834,  46350487,
        68082478,  67205708,  39661299, 135554529,  49733265,  99617408,
        28573874,  14650147, 111052616, 136950449, 151032795, 144065682,
        24154768,  87504658, 129877613, 140310297, 127020588,  51366780,
        59501524, 135917561, 118149037,  78923982,   7535185,  70997237,
       125147881, 152780945, 130311218, 127650635, 105373274,  93021706,
       149628563, 150195986,  58331059, 118327672,  46808114,  79200123,
        86903643,  40816249, 129268658, 121275411, 117581888,  90237544,
        70632458, 139190360,  38608953,   9788360, 125158973, 138218395,
       153079983, 128408958, 153865604, 125432328, 148590269, 139716234,
       123597955,  68374050, 124998496,  70861859, 151742528,   3665090,
       149136385, 140729369,  39410323,  53874552, 137378439,  93922966,
       119366536,  52616243,    746300, 143396000, 127412361, 134301350,
       100140045, 100006466,   6806781,  99229563, 129127263,  55291764,
        96233522, 141877430, 125954080,  42295262,  66350605,   4305364,
        87959930,  31597770,  94110351, 117864730,  38932394, 119346630,
       143129231,  77499409,  66515493,  89008832, 132132222, 133765769,
        23042724,   1261465, 140425151,  27040241, 108271378, 133335963,
       153836179,    166683,  63496188,  76291619,  60902914, 110550393,
       141307430,  83143054,   8646211, 144782713, 140897745,  20700771,
         5335373,  60851325,  40177198, 136466469,  88128130, 136651435,
       124852885,  94385825, 153313564,   7069880,  62549198,  39750878,
        12456377,  96797188, 110410141,  95208727,  91290219, 126248664,
         3220144, 145066983,  91690522,  74512391,  63246684,  56592845,
        65761368,  66643046,  51005889,  50615155,  99259159,  99255837,
       113256025,  73640446, 137314408,    851930,   4275447,  90078141,
       139678994,  28953769, 130272022, 148162675, 133845979, 110621071,
       149746648,  12396906, 134326647,  98512214,  22542582,   2308404,
        66454295,  10696546, 105152284, 128018318,  85512121, 118604972,
        25727137, 142578237, 131554179, 124453359,   7386882, 137629305,
       137270833,  75740328, 142093293, 148315253,  57723242,  75758711,
        63249336, 151047983,   8629501, 125853738,  33230361,  93720704,
        94393493,  85645015, 109354546, 143407633, 132050778, 129341222,
       141638882,  61488178, 112006652, 153900148,  91172315,  38418232,
       148684964,  33380476,  54098761,  48634899,  71014470,  37117124,
        60834468, 128014249,  79103158, 103157673,  94465238,  44125791,
       130835296,  72246325, 133004014,  81080267,   1364753,  63467887,
         6190678,  95825417,  21011025,  30335217, 109811066, 146362674,
       103268073,  34923944,  72133718,  83982917,  61339778, 140121798,
        78902916,  55072369,  44515586,  71472448,  68369335,  75232119,
       146162531, 128381530, 147399211, 142697207,  73734939, 121335370,
        26830151, 148049507,  93995873, 143485050, 132128658, 136394119,
       146394099,  10097814, 112339368, 131354912,  84496597,  60764194,
       143451424,  23824900, 131634831, 135646048, 137683233,  65634374,
        98237389,  75182178,  44935416, 135803603, 136810190,  57694541,
        85805732, 134062262,  39439982, 111609508, 105157744,  63580412,
       139432103,  39670583, 126410690,  60365369,  76402093,   1623202,
       105895465,  69763195, 111551510,  76437729, 105557168,  51065792,
        96650188, 140495188,  33641843, 120925154,   6577253,  69679341,
        23973716, 112156265, 150522442, 127129375,  94363068, 113893331,
        65453544,  22382218, 141802346,  89788907, 150143121,  40694224,
       130782643, 147769409, 148170118, 114517326,  71734250, 107351472,
         4544935,   2083226,  62336474,  60326136, 129583587, 126903726,
       150081597,  72682970, 126520809, 139303757,  52026300,  88615294,
         6514936, 126367399, 143168532,  27500383,  60840228,   3066769,
          496226,  67378462,  60706736,  74723434,  56421320,   9704635,
        70863880,  15690716,   2234756,  98967001,   8494844,  58832685,
        63004280, 147612326,  75333952, 105944423, 146781155,  16762228,
       128582014, 146364748,  22352863,  58131444, 144466094,  21450537,
       130948397, 122421007, 108311486,   2226965, 140563041,  52939191,
       135844646, 150110861,   4278276, 139210947,  51126342,  91291196,
       138248227,  16302294,  57176468,   5455040, 146022752, 121355920,
         4301324,   4546506,  78130743,    810408, 100116069,  85585026,
       131716886, 101538236, 109707493,  84929901,  89908266,  77321395,
        24616348, 130665419,  96615211,  86914338, 119361194,  95547313,
       119239816,  38292138, 154184538,  65803305,  70846752,   3664793,
       143139621,  55677718, 153933162,  27090174,  87844351,  14589122,
       127239795, 131005581, 148835906, 115457371, 129663212,  61489671,
       138765951, 118659452, 148672078, 153689132,  91002999,  58240823,
        74493744, 118329133,   9209574, 138725324,  46988603, 121272728,
       126461707, 146784098,   8185502,  91596266,  95325063, 119265628,
         1172830,  57264900,  76223961, 144179698,  39389607, 150182577,
         4279597, 139341790, 147162259, 126430910,  83212875, 102105689,
        97904208, 110746819,   7701495, 131277938, 145218860,  91534765,
       104161145,  83418839,   3234960,  79692291,  24646328,  99290408,
       137199379,  74662307, 100458233,  17293223,  38609699, 152421974,
       128211985,   8587305,  43185248,   5780507, 152364263,    875198,
       125008021,  68752181,  17734870, 113032323,  13418771,    415585,
        92450938,  33241419,  68366909,  32089497, 126160530, 143437008,
        52076342,  60665141,  18554926, 123013121,  77566523,  70975661,
         5781087, 119951024,  38657174,  93910877,  42353491, 109101727,
        17422773,  93337808,  60276345, 111531363,  67868718,  65345032,
        54598285, 151371448, 145064945, 113327520,  58170538,  31547315,
         1383586, 130914425,  91299391, 147069527,   1381910,  46671263,
       151249057,  20780600,   1417274,  81841182, 101432977, 144797851,
         1495892,  85602739,  94004243, 125681388,  81553385,  89837102,
       130170126,  39701367,  77576874,  92982552, 114227927,   1578405,
        57673368, 149519528, 140974614,  96730837,  15251043,   7006130,
        94170286,   1060597,  77459836,  43550413, 145114218, 145578876,
        38606450,  70692535, 102905568, 119376024, 136916434, 134894959,
       147601046, 152939756, 129663599, 152761304, 139497208, 133359451,
        73529590,   8885776,  33400588,  75223933,  37720268,  89966925,
        60813667,  79632892, 134410744, 142876047, 127444071,    734018,
        59709202,  68539255,  60980329,  58141396,   7564708,  65434500,
       126674361, 126800425,  65512992, 136598555,  93247532, 144225762,
        95120185,  49654041, 120313602,   3064942,  46735774,  40942672,
       137718547, 126194987,  83806656, 137969740, 118285034, 108471503,
       151116599, 134786363, 110730198, 148344979, 147804084, 150059515,
       144752460,  55707149, 143216438, 145804173,  63064551,  60812676,
        54157429,  22942588, 111886966,  26839930,  87053620,    626581,
        32158251, 139003260, 121893491, 146581488,  10635155, 111786993,
       141078783,  19057393, 138979750, 115385882,  96921887,  69702544,
       128411030, 124947351,  51857953, 135102522, 152021479,  39600256,
          508967,  81476836,  10065931, 153065580, 137353896,  43500720,
         6176929, 136909229, 134407161, 138341533, 154090648,  59945820,
       120040898, 118272027, 150654276,  58653550, 141570575,  88781151,
       111671624, 132893945,  74623415,  33360135, 136602125,  76412533,
        88122799,  75954359,  58573302, 100085981,   3634251, 142008354,
       121309013, 121738758,  77507391,  61330091, 130286999,  84515512,
        57703350, 119307209, 129267808,  58553816, 151061320, 150390432,
        68957716, 130839344, 136781787,  19336335, 111488425,  74622139,
       132019783,  91793292,  40942279,  11955653, 126691985,   1184916,
        44815549, 127606066, 113843490,  43720150,  76436123,  79064969,
        39523253,   3297985,  66446192, 151746617,  55930344,  44469473,
          996814,  97903200, 149553548, 147292057,  22413666,  55140590,
       135878585, 130129964,   8285363,  87864286, 130692734,  85699404,
       129522559, 152540155,  95249851,   5038368, 107216778,  52636136,
       147199479, 136011774,  40571615,  91395413,  84493535, 130662966,
        97759331, 136083898,  43440405, 126813425,  88074508,  73518125,
       145385422, 131017422, 103470084,  62327749,  58170935,  55311720,
       101881246,  43649260,  38445418, 150390237, 130537522, 137224397,
       135752067, 106336465,  23313321, 147078630,  25937355,  61853347,
        49136217, 109344317,  91018004,   1205697,  68377610,  58524455,
        48171498,  79191013, 101292964,   4394673, 147593848,  62744526,
        54969046, 129586549,  46204135, 151850238, 149768244,  22283495,
        83686831, 116679249,  97860555,  99886803, 153206476,  94232267,
       127964173, 119970317,  34753593, 150609685, 132827483,  75798626,
        65048270,   7376204, 153406279,  50576921,  46336643,   4289694,
       143525904,  34302841,  38609280, 138387072,  34513700,   4277572,
       147522028, 127182893, 138380261, 112080221,   1824859,   5937358,
       116979676,  91242850,  54344901, 102215791, 130511877,   7369546,
       127163786, 136101931,  71744897, 144186363,  60811180, 130313703,
       109272418, 130916855,  14609037, 142177703,  38536131, 111646705,
         1576567, 110432636,  78859331,  65079028,  85584642,  20097403,
       120533209,   6287063, 143070744, 134102399, 137437540,  43409757,
         5176029, 153222562,  82860115,  93281122, 139283679, 121919316,
       134082525, 137320349, 132880070,  70711141, 139711565, 138263160,
        91641089, 151001239, 136431650, 135046918,  70081991,  67356998,
        27491469,  50143695,  70711198, 124825849, 112119254,  54533271,
       141806308, 129313350,  61449512,  55917619,  41061401, 108865954,
        50094212,   3694822, 136291951, 107687614,  21230628,  76052092,
       119269612,  34622937,  92103842,  90651565,  17634714, 109716127,
       125486683,   3289655,  12496954,   6520872, 124725075, 126080177,
       138146008, 122206758,  94361658,  77554207, 132507808, 150618691,
        83739991,  84526976, 111074891,  95098519,   7286760,  85711776,
        60872836,  80540147, 145445922, 143463912, 119230139,  62297800,
        87042616,  17904919,  58250239, 109233359,  18605923,  34723436,
        61470616, 129112405,  20811889, 112007562,  36341617, 121804099,
         4525604, 109504699,  74584400, 151145684, 142118567, 117205503,
         8765205,  10548963,  70983655,  42275098, 107216265, 135015749,
        59151177,  75129257, 121312693, 151462166, 133275624, 115416358,
        68984879, 151504046,  79752618, 148013394, 120020893, 121294342,
       108235600, 121430313,   6384738, 132729017, 126371782, 153528921,
       143088642,  68393929,  81355572,  65797289,  58512628, 131503826,
         9744740,  50043673,  97435776, 128975087, 103961515, 151185531,
        12705436,  91031670, 103218392, 142878907,  23834958,  18874972,
       144434716, 148974713,   6788344, 117199243, 109524351, 152747411,
        72857416, 131230452,  29023913, 117871394,   7047862, 134495936,
       132880228, 129548673,   1060895,  57194598, 131425074,  66122034,
        65098690,  41113451,    770980,  97779756, 146809563, 153214267,
       132916518, 145763227,  56130187,  73569289,  46810346,  91564029,
       122693391, 149440723, 146566933,  37217474,  94081601, 132130521,
         3150264, 142614651, 110712996,  42404257, 152480258,  95086254,
       107695897,  65304217,  85420154,  69772908,   5945086, 146643606,
       146639529,  77470329,  46255422,  99586222, 109340128,  61439514,
         5844836, 151054471, 136066167,  60687055,  88008943,  34443638,
        61492660, 145697363, 124748499, 138485310,  12497677, 151413183,
       140960650, 121794901,  71033441,   3294511,  75164442, 111769127,
        16682194, 148248415, 114170852, 144744044,   8972039,  44775954,
       142227359, 148678803,    455618,  65797443, 130057446,  77340602,
        76363828, 150045449,   6299438, 102554235, 127267639, 113854338,
        68397074, 132314338, 138948674, 120926459,   9055474,  93483931,
       150818598,  87094386,  28662984, 142573681, 131498702,  61453052,
        52026128,  27591610, 149368715,  55968566, 115384262, 144649815,
        88129481,  68604513, 134812185, 136636463, 119231518, 142661212,
        54117713,  81456657,  60477456,  58503483, 113741115, 147314381,
       104151154,  88862560,  63970518, 141283225,  76182468, 136457597,
       101522409, 129043060,  16321871,  64067274, 131955904, 126838400,
        19997382,   1494918,  78962804,   8961395, 118517619,  87968107,
        63415995,  14197786, 121131351, 125023775,  13197888, 152690415,
       130083503,  91357870, 134461428, 131659992,  45324392, 113845393,
       130098383,  66413903, 147946459,   1173242,  55545146, 139654290,
       141996887, 121322751, 140799028, 148883600,   1214183,   5704703,
       132938290,  17695320, 135402375, 140379298, 150068273,    348005,
        14520851, 131842378, 113122743, 142989489, 130000877, 146370710,
       125627016,  97152125,  89988796,  60145483, 120592729,  85539697,
       124000224, 145376372,  66025645, 105021251, 132308615, 121061757,
       133441361,  65692801,  77317655, 144775644, 118167068, 152866946,
        94991552,  94429913,  75758485,  74551771, 135762165, 146111645,
        90389289,  70121615,  23996032,  60235800, 105676162, 135112612,
       150470306,  46377741, 145669257,  63287965,  11774717, 142667373,
        43789637, 143465268, 136383199,  48080620,  88983705, 104102855,
       102085723,  91223022, 148423876,  37257631,  90054783, 145191575,
       126759915, 113876310,  61538406,  81900404,  88784480, 152271198,
       146918946, 134502547,   2837260, 125916847,  52057521,  60900970,
        62206223,  94446285,  42974235,  73865551,  86593298,  40472143,
        94212088, 134421311,  27410096,  71846063,  10159106,  89794620,
       106062992,  14628304, 111702374, 133476596,  95170937,  34963745,
        96749904, 138230360, 133381534, 146244244,  95143441,  65884362,
       110380757, 138159873,   3486084,   4557985, 151494132,  92055303,
         1578742,  72554528,  65854298, 118654397,  89848652, 150219097,
       110669049,    178446, 151898121,  15640417,  76202705,  74685525,
       124838272,  16342143, 142110496, 152210863,    881055, 123029064,
        47592968,  91964539, 126104375,  58400129, 127317400, 107540588,
        51196758, 125940802,  16041350,  71056773,  97426117,  65895328,
       104213840, 125799148,  75800067,  60385631,  59581717,  46386310,
       100456765,  46803967, 143486915,  68475807, 113800857,  13008721,
        31657771,  42984302,  91425528,  42444932, 143373339, 146039616,
        89917529,  10101299, 145582463, 116547999, 135353661, 145271251,
       129821691,   1600290,  18654850, 151624949, 128833267,  41161919,
        78510267, 124887572, 146557718,  46846190,  36361038, 107570368,
        57316821,  31296588,  32279037, 103308342, 130510391, 111876202,
        32368792,  68960080, 115687516,  63158925,  69136183,    991964,
        70921983,   3238694,  70702994, 139187341,  12479156,  95353567,
        75769296, 145485894, 145470232,  60635780,  96187770,  95365352,
        86116726,  77005933,   3665945, 121227170, 125227101, 126782704,
       124916352, 123058343,  94275372,    223015, 111669637,  91089763,
          680409,    114469,  72844267, 124071626,  64241005,  61513708,
        88861087,  13428108, 122313710, 104981618,  79752353,  10148958,
        69573085,  45394417,  12496861, 149836000, 129249938,  90991859,
       153775466,  88860574, 109867396, 138149539,  48001349, 104300198,
        75890322, 113863878, 149979113, 129764586,    249699, 154071689,
        40217481,  61372072, 137803920,  38444701, 144099300, 147773503,
       102256278, 144899394, 148679299,  95641377,  95061062, 142952947,
       143211166,  18506112,  94341575,  94161135,  69425037, 150453244,
        71837511, 120120051,  66582549, 148963096,  85227448, 109341516,
        94100648,  78459604,  79602560, 143079141, 147055888, 126163527,
        68367142, 100162517,  70600590,  92331553, 146784228, 144225687,
       106661491,  52027234,  28633301,  79073360,  66651090, 154106167,
        45172131, 145597655, 139489895,  73620777,  44816120,  36089036,
       151867530,   3289005, 116267587,  68714191,  14588794,  64893365,
        44757485,  75060871, 111472631,  96326409,  21021110, 116746243,
       136448659, 129171440,  59291109, 145042206,  82367379,  98104149,
       146576246, 143653838, 136821447,   7616375, 126076289, 139146152,
       104362187, 147708505,  53494203,  71826161,  79091317, 115044761,
        50093482, 110685600, 135840114, 125549065,  77448169,   3639616,
       102744205, 132451175,  97872552,  32148649,  45049392,  77180335,
         1129440, 126818201, 138725838,   7925956,  12446859, 135918417,
          676800, 140985983, 148604574, 109322130,    647537, 139318957,
       125999152, 141012717,  39559527,  79340735, 139183608, 139533755,
       149627746,  35948634,  21461004,  48554978,   3733014,  82649244,
       124836688,  94299913,   6394695,  10152804,  52199321, 116503326,
       128656667, 138050096,  68340396,  50223552, 135437925,  96910032,
       109345446,  69652564,  49158630,   8616971, 131484437,  85260684,
        58170816, 106508619,   1102764,   4314590,  67335546, 129336119,
       136153616, 141493046,  46673506, 144515037, 147931332, 142315031,
        98438518,  30305599,  21370700,  81493495,  35104265,  31197307,
       144545145, 135544380, 127695976, 106326327,  29783944,  75243498,
        61471961, 127192074, 129964579,  23213105,    530253,  74031692,
       104016672, 140965003,  73735391, 127417196, 132614545, 138264944,
       148578598,  91743795, 153901266, 125291734,   1676292,  33281182,
         9877450,  74122276, 139034366, 153125311,   1693177, 147482297,
         3016641,  79865764,  49955747,  85624816,  17432968,   8306346,
       146277784,  26097985,  20079568,  58140375, 144553277, 135537632,
       130875407, 108003579,  41860540,  12946944,  80658423,  88975493,
        78409470,  94225844,  13098221,  67337272, 110805333,  12955943,
       134149314, 128720439, 100113983,  98012681, 119962887, 139940705,
       148076528, 148573151,  85521445,  43195135,  43336131, 125827735,
         3153958,  28624011,  44653831, 133376004, 120620782, 131130274,
       106952682,   1531252,  10158687, 107470241, 120051515,   7378024,
        63489850,  87354291,  32319571,   1090540, 150566173,  73924938,
        91345586,  31828118,  55933554,   9197061, 126612283,  40921255,
        84596245,  57981244,  78819943,  26007173,    659790,  65637595,
        72675366, 115106777,   4425625,   7094057, 127919527, 125088847,
        70923176,  74876323,  57684264,   1303814,  90408273,  63448930,
        55170850,  96291884, 120141307,  95223023,   1190142, 142838242,
          470131,  55544882,  65643228,   7056507,  55948999,   7904991,
        73468779,  64280548, 123971771,  61408964,   3917632,  67788243,
        85046264,  67859981,  19607364,    572870, 146576786, 110034606,
       112725296, 122268636,  39742724,  61932503,  73968342,    773935,
        83750191, 100014284, 121742916,  91294539,  95651882,  90394518,
        68912939,  59201222,  29805630,  59081316,   3221594,  72633734,
        56441263, 138953647,  89935974,  38605064,  52959880, 142372051,
       126786275, 123612775,  91958647,    516551, 122310668, 138393153,
        41419146, 109243089, 139077048, 129557425, 137040315, 137526626,
       137521638, 119234983, 151987948,  70071215,   4558857,  90149395,
       135681095,  60954885, 120933898, 120616670,  51177011,  18844944,
       134352966,  12585276,  62714684, 142044974,  56029744,  12415937,
        81463568, 125827338,  60685522,  70082468, 102524550,  55411588,
        37731870,  71794234,  36510604,  63064246, 145424649,  98936368,
       144838302,   1437527, 110665117,  83859773,  63154897,  12346930,
       145293312,  38476446,   4376310,  88813692, 123041902,  40866933,
       145728052, 112749813,  32218516,  94165155,  88959812,  96420086,
       127748678,  77148746, 109907289, 130106177, 130838321, 120012639,
       126744214, 120179267, 120657268, 119156745,   9777388,  92164205,
       138871465, 145349026,  18545799,  59551620,  60105534, 139323293,
       116879762,  80558191, 117191716, 131394117,  31156995, 134982076,
        88077487,  42423575, 146743048, 134768883, 133003142,  68527148,
        43255440,  65701980, 138525230,  55309660,  63429682, 132185475,
        90906450,  73774101, 151836347, 140265432, 149418122,  91029782,
       142042518,  91684758,  29974761,  69903223,  28682823,   7382108,
       129517257, 100095527,   3488159,  75141886,  96508610, 112596406,
       126061426,  42444906, 117361814,  55340515,  40421185,  65804152,
       144726431, 128735598,  66580499,  85734644,  66525144,  31567715,
        10557228,   6647182,  43779340,  49036261, 153364330, 137843636,
       137403627, 153379150, 150801691,  59523167, 134532417,  69085186,
        77168386, 129688439, 118233723,  92114655,    432311, 137857167,
       143861356,  56131862, 153589759, 131067622,    284465, 138841462,
       149478990, 109976749, 119477383,  99974866, 125181987,  12705385,
        22283555, 132777371, 118263247, 109679349, 152223318, 124848414,
         5945409, 120138986, 137129822, 130057959,  95341223,  63752456,
       139935543,  24033847,   9817079, 143846250,  13637292, 134988118,
        93114940,  12626947,  83954048,  92391042, 131333406, 151547502,
         1430797, 149802993,  56027163,  63335498, 109802963,  65068610,
        55218370,  66535553,  39887424,  90248217,  69565102,  42284294,
        78478388,  65843380,  70731921, 139876591,  21230939,  40931649,
        87804366,  86986294, 153468122,  91609428, 113922243,  19176365,
       151413282,  75859259, 143316198,  56874511, 127374376,  88184582,
        91633870,  96295318,  75690131, 142046384,  48020229,  75838838,
       130092077,    654557, 148159950,  43266130,  54248315, 133938367,
       150199352,  34332122, 153693718, 131549150, 119307274,  50428032,
       102674482,  53654726,    636990, 111105050,  85522081, 136117248,
       145061414, 144298365,  14839152, 153886827,  77111158,  64079295,
       131384063, 137202336, 147114411, 132425825, 153370432, 143734838,
       143838876, 144846753,    504550, 139188479,    691393, 138575984,
       147994823, 126276133, 132767950,  97569776,  74721304, 133200926,
       106506484, 148234966,  60791751, 118152569,   6286913,  41600161,
       136487190, 139615334,  81415749, 137041858,   3527465,  35338168,
        48050279,  68895728,  56102405,  70272103,  95022211, 111101831,
        96243361,   8136828, 125223365, 142578912,   2084490, 140265812,
        44503774,  13117302, 136117503,  77208554, 148523226, 131578105,
        70602645,  60981396, 144789255,  97405916, 123010627,  50525562,
       132481109,  38604251,  21851232,  94250334,  56129949,  38568237,
       141305337,  10136763,   9185215,  84604155,  50034383, 151294231,
        67646059,  56107613, 139210567, 128980165,  28693368,   8590002,
        91823175,    416011,  70150314,  56009016, 119985331, 128554498,
          705233,  19127400,  79113243, 139152541, 128372375,  67585280,
       104123012,  31688070, 142767377, 144266723,  62296470, 153487431,
        36251637,  34001789,  65641813,  55149089, 126567507, 106204107,
        57894328,  65422220,  60740262, 125519162, 153318288, 119971687,
       130407226,  60894420,  65048587,  15570406,  18625418,  38480297,
       151667971,  70996669,  48636254, 137363250,   5030983, 143002618,
        68577412,  75870810,  72555887,  56037637,  65352621, 126360703,
         5801439,  53674115, 130961445,  78933512,  68881576, 149679117,
       147856410, 152691450,   7535187, 144714770,  42763134,   1063199,
        76073916,  90135116, 100459711,  98179765, 121923854,  57694011,
       109851517,  72554363,  67665931, 107069093,  77070505, 124722703,
        17633466,  93913027, 111129407, 124460590,  65627156, 145204973,
       141879449,  99926044,  54778745,  73400136, 126175352, 152303373,
         3605455, 132170709, 144105972, 100132134,  59221302, 142809798,
       109524931, 129762903, 149074754,  37820816,  95665497, 139220901,
       147682329, 126432908, 146572507,  55290500, 112556690, 152044545,
       118225377, 107089209,  88663873,  61992850,  41781124,  39519419,
       153140983,  48714411, 120242450, 152745796,  15299733, 148255144,
         4301728, 101137726,  56117386,  88694473, 124427761,    738877,
       110520518, 141045925,    813303,   3930409,  60396349,  62033480,
        91533540,  77561073,  94254114,   5782986,  63928105, 111751637,
       100023666,  64078179,  84574605,   2309038,  23813950, 134071148,
       113317680, 154134080, 131464953, 141467750, 134915430, 124100449,
        58351171,   7370068,  54334365,  15470789,  98125701, 149610710,
        55361745,  61364181,  18865678,  52768252, 112807655, 134406838,
       120257406, 123022071, 129899709,    499397, 111099266,   5344801,
        76262717, 137197693, 126813553, 144650272,   6309050,  45324933,
       143440003,  69976229,  78328419, 127015323, 126954414,  73490892,
        77805337, 131521212, 111956557, 136736393, 142924798,  85481510,
        59461425,  85138269, 113406964, 141062254,  79111805,  22481879,
       118617621,  54304371,  59452681,   2935735,  94114098, 149903910,
       125882651, 151815995, 110383697, 113296094,  43076362, 138495796,
         1249326, 129041544, 151901304, 121431087, 137843618,  81386702,
        60904875, 139644234,  58430323,  86994662, 134614720,  65403954,
       139113579, 130296285,  73833278, 108235799, 121907011,  41110982,
        20712879,  25167256, 144458366, 144174998,   8586139,  90696318,
        92339669,  91223043,  29735050,  62186605,  97839892,    560669,
         1399387, 122308009,  40402005, 126483625, 109044216, 129750154,
         6597003, 100035356,  63803474,  70937150,  38418078, 148643436,
       114914272, 132637852, 143734610,  48736214,  78180016,  67615354,
       109325562, 145646300,  68518773,  57216982,  65304757, 116828328,
        13116857, 117859589,   2311829,  37157224,  98438750, 143174224,
        30235922,  74611910, 148028397,  48444456,  41161162, 143298430,
        70951765,  59935164, 130752043, 132194156, 146366263, 118388353,
        78869657,  46804046, 130539744,    504238,  84820160, 135952318,
       141185058, 135769558,   2974706,  19266181,  65302964,  20027848,
       145525012,   7686270,  55433179,  61333184,  40919348,  75299207,
       135538247, 111082846,  55544631, 130539646, 131546848,  88096365,
       139575636, 116249871, 150347946,  57166205, 100111952,  29574734,
          679069,   1190450,  67477887,   9854671, 122321671,  83272447,
        38519739,  67428470,    768056, 135346144,  75597817, 135683598,
        43359697, 135677858,   9844577, 144700044,  12856288, 148466345,
       138283420, 112063426,  74594651,  66170677,   1446277, 130865214,
       133222031, 136307379, 118193739, 132099945, 120082246, 150216142,
       100438126,  88493493,  14678749, 141688721,  88831556,  17914826,
       149339284,  73333210,  88826567,  66473088,  97859666, 121221463,
        13478079, 130238099,  93970289,  10081974, 110101326, 133312827,
        65372837, 127065036, 140670481, 144810362,  51868752,  47210191,
       128311763,  65657180,  55950054, 150142050,  74795731,  14558753,
        98042770,  22423604,  61521992, 107236554,   5084673,  10638596,
         1008914, 112967223,  92749290, 143971089,  87136386,  90057115,
        62599716,  61502240,  15290320,  94175501,  36251514, 127506908,
       135082092,  72703600,  90521364,  53995494,   4445034,  77469072,
        10147827, 103418715, 149197831,  83518286,   3626698, 121277378,
        53031003, 129935483,   5004807,  73701586,  68851226, 143442467,
        91427771,  83839905,  69614922,  86176629, 128534911,  36340139,
       115364585,  31467637,  25075971,   8987380, 126055374,  73905836,
        95072044,   9755211, 151880954, 132813863,  65625502, 144053826,
         1601195,  44065492,   8636052,   6898389, 149171284,   3154133,
         4435893,  59231438,  41090498, 152844234, 133441517, 132911242,
        97277613, 134415267,  35949117, 148265570,  50435598, 127364384,
         1681914, 130306868, 109339583, 151869061,  67635364,  40402712,
        62285500,   1604639,  92543442, 107661806, 151999698,  51366803,
       137530053,  97865338, 142647719,  31867884, 149946695, 126700713,
       142267379,  75477519, 124032213,  24154741,  15690961, 146900662,
       132239910,  85672916, 151524447,  57277179,  80589254, 149802089,
         2377118,  78779797, 134703540,  88271245,  21230024,  27590932,
       102594344, 145518047, 128216192,  96354940, 111080980,  76834500,
          893118,  70955354,  74855038,  68713204,  49963353,  38570567,
        26340752,  95638181,  74091784, 137601829, 145226640,  43945368,
        11985992,  97288739,  96955987,   6344774,  60586928, 149364989,
       111646571, 113876119, 113868288,  77952043,  16862610,  40278432,
       146308258, 151522936, 146958004,  18496216,  41309258,  53322773,
        37207310, 147090200,  79035262,  65843169, 121728306, 101483038,
       125017519,  85251546,  43819759,  63177035,  64251032, 107658362,
       125286574,  37147834,  68987148, 151817122, 120066959,  56009710,
        96667366,  74693768, 144564497,  96628039, 113201851,  89036135,
       114151749,  37701521,  48534272, 151358707, 113001273,  65170723,
       110291592, 134601840,  71845907,  29715253,  47990694,   3240028,
          534313,  32028257,  72082797,  43490319, 116523227,  10147047,
       110620247, 109651922,  18354556,  56572420,  27731890, 143873157,
       150140601, 116269287, 136303206,  60655755, 143912479, 132577707,
        66290465,  63967527, 153560799,  55535127, 139086313,  46652690,
        96487409,  13287024,   7089822,  21430546, 113848491,  60527137,
       128234647,  97388248,   6206886, 153794240,   1104582,  22392420,
       132076750,   9207772, 141066548, 141807590, 122356836,  61442816,
       132370518,  16482107,  54335649,  68517934,  83858193,  94329234,
        69862487,  49177548, 113205551,  51986292, 128130585, 146301847,
       129172277,  63285789, 143855588,  57156606, 146789351,  70190229,
        63177672, 127091322,  70932358,    821305,  88583138, 109644141,
       149048638,  12406880,  83468744,  67725933,  49883208, 126127958,
        54101547, 149949293, 110141878, 109192937, 130706204, 148632329,
        72203855, 121866201,  81484975,  71453242, 110798139,  94461149,
       146717726,   6614835,  60970904, 153804606, 149656119, 144292981,
        29023743,   1472290,  19246263,    743088,  40350963, 150955169,
        79601624,   7093486,  76183162, 116919308,  59443440,   6310179,
        51906990,  96938405, 109817735,  47169054, 125626932,  16141224,
         7726076, 147689580, 153506680,  85539152,  99904314,  11418600,
        85540966,  69679756,  43500828,   8600386,    853805,  44140860,
         3922007,  94157611,  38098311, 121738815,  46558754, 144581321,
        16251106,   4309449,  55148285, 141967457, 116014511, 109847776,
        34353288, 136446607,  98136631, 125251245, 119690561, 134525884,
        15590808,  17283475, 124463494, 139083985,  65384340, 130565604,
       134672556, 126643345,  78188996,  13417760,   7076539, 133217912,
        83123482, 126429909,  48555068,  99828835, 143155400,   4154570,
        77407559, 118314409, 124426064,  64009777,  57234896,  59189263,
       146346734,   3150694, 125973229,   6542435, 150222095,  39419238,
        55311279, 114227855, 113026286,  63418071,  98885757, 131982078,
        67015350, 150301375, 128569985,  84124401, 117915003,  63507546,
         3157379, 137697982,   5617737, 148702434, 115527174,  98297584,
       142168220,  37832099, 133871980,  60295894,   6719232,  86091133,
        95190048, 127543042,  23333475,  89613166, 122976043,  72754785,
       147276298,  71563145,  57215068,  37721869,  69655150,  59799266,
        13347315,   1557378, 111975708,  95200925,  84799763,  13028327,
       145866891, 102350470,  69514592, 135699299,  29805210,  51326124,
        85659603,  67328269, 137813987, 124914524,  91536404,  75467614,
       103238239,    546088, 102674798,  12509367, 124997944,    395093,
        73420624,  66461507, 143120914,  79511461,  96325585, 145707936,
       127318123,  20419000,  91507760, 126172099,  46802444, 117844364,
        55180743,   5792384,   1193709,  26320258,   9068098, 143809845,
        55717280, 119252610,  75143548, 148096425,  30996148,  66016320,
        73438600,  85591240, 111511866,  65606508, 134218046,   5759136,
       150367221, 117885255,  94105571,  28062160, 139484938, 112436798,
       137721005,  58902503,  59659368,  61523227,  59709456, 143456433,
       120136106, 147992390,  66443287, 133042500, 134686758, 143607949,
       124849649,  48425131, 153216823,  39239876, 109670178,   5865820,
       140804182,    439566, 129080052,  68675158,    994468,  55180628,
        17993974,  62539403,  68376710, 139020794, 101961492, 139005868,
       148410487, 138062816, 113226120, 144916713,  83669961, 109938852,
       109995919, 129688702, 129940962,   6697006,   9858276,  63469706,
         8971698, 146249233,  46274965,  89977350, 136249116,  65806672,
       147627828, 139366204, 114224207,  99596750,  49723213, 104911887,
        40510340,   7634942,  75369738,  36300708,   1147196,  63793618,
        71418641,   6935379,  88493744,  85530491,  40060877, 145758857,
       140291143, 134083329,  91375248, 132028083, 149936960, 120256617,
        38628433,  64903561,  57813573, 114133833,  20869334,  73763145,
       127012312, 146255746,  25807012, 146618714,  97550343,  94021431,
       121329965,  92598187,  68633874,   2275591, 143589288, 132312944,
        38526158, 103647971,  40531383, 112456824,  72259208, 131957858,
       124797684,  60812066,  85208334,  64681717, 109317584,   1465735,
        92523509,  85713136,  54638629,  19597062,  69226365, 116718045,
        50527766,  88175707,  65786868, 113877437, 107489271,  88195300,
       102564099,  53332316, 150924219,  85568019,  60557139, 137774019,
       139833075, 125554848,   7052295,  60772104, 145697808, 147466255,
        65936670, 140814265,  22792876, 152974417,  61792303, 116822733,
        48634737,  74955405, 127901345,  15550571,  93475157,  76332874,
       145986042,  55160713,  73488945,  60565817,   3292381,  96567765,
        61390755,  28332436,  90755461, 112807628, 149934949,  31116156,
        39228239,  83204438,  22332959,   1430752, 145744763,   4796894,
        85637266,    448303, 143544754, 134876182, 135431582, 116808093,
        43509946,  70601185,  67337481, 113549035,    429634,  87962815,
       136691064,  29234521, 130428865,   2309865,   6736166,  91043122,
       120562819, 139969364, 148161674, 135255947,  98694351,  15339899,
        72663317,  10756435,   1344102,  96486072,  69060133,  18536000,
       131031750,  75211016,   5771102, 127527704,  32289713,  16041606,
       122661873,  41490637,  27130277,  39620695,  74211616,  52038245,
         6570182,  85718945,  78239373,   8627959,  85640380,  75710792,
        68603612, 134772155,  43957246,  96317437, 107350452, 120563819,
         7954926,  88865841, 153206948, 134330721, 151303056, 116829769,
       139497767, 115380093,   6727723, 133211520,  58141389,  94263011,
       122294733,  37792120,  37750380, 140679875,  67238577,  35196200,
        96677461, 140802505,  90341313,  79091831, 113075932, 129923009,
        19696898, 109181925,  29013725, 132180603, 142119003,   1481820,
       109848008,  14659759,  90379234, 122249996, 143906794,  17363018,
        91152875,   8324659,  85186256,   7046099,  54443794,  97561002,
       138531637,  64390822,  96379450,  73763680,  57205035, 132477735,
        99687323, 146636411, 134995209,  85617284, 134248141, 143042438,
       121905609, 119273331, 126456090, 109544590,  84700755, 129267333,
       103637944,  13127646,   3234702,  78799418,  50536887,    353294,
        70752435, 150442422, 108781675,   9035966, 128039238,  88271496,
         2092451,  86081121,   2306726, 121915125, 149521390, 122247005,
       114942933,  78649507,  43560058, 107026115,  74965252,  51908718,
        45584321,  76011533,   3635535,  85530889,  51636397,  65926264,
         3728770,  19157000,  65824301,  24164834, 104342809,  13478302,
       140444615,  10078668, 117232152, 151853176, 140798210,   5777372,
        43570627, 119280288, 114171734, 107091326,  38598870,  91157682,
       150142894,  56421691,  34893306,  42484992,  38371423,  96713097,
       141124425,  96533797,   1209355, 110660023,  91593132, 118607955,
        27702358,  68694143, 128719469,   7076576, 113050208,  85570416,
        52819994, 102794678, 139759374, 150984797,  28923445, 135836164,
        87985772,  36371531, 144974492, 134203250,  72614065, 102076927,
        74643104, 126355179,  92607378, 103237564,  41196658,  32358817,
       133928672,  50655139,    826376, 102675920,  46674080, 146151764,
        94546730, 141831418,   7946220, 151516744, 120273895,  29174520,
       119477552,   3295183,  51075576,  84665326,  44025421,   8836084,
        69981997,    732729,  75417346,  93464614,  73141985, 151538647,
       147157181, 142579383, 122310665,   1003699, 146404710, 127841437,
#        128147838,  71512586]